# DX 704 Week 7 Project
This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [1]:
# YOUR CHANGES HERE

# Import pandas, numpy
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Import Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Get the directory path
path = '/content/drive/My Drive/Online MSDS/MOD 8/Project_7'

# Import model-A.tsv
model_A = pd.read_csv(path + '/model-A.tsv', sep='\t', header=None)

# Import model-B.tsv
model_B = pd.read_csv(path + '/model-B.tsv', sep='\t', header=None)

# Import cost-Q.tsv
cost_Q = pd.read_csv(path + '/cost-Q.tsv', sep='\t', header=None)

# Import cost-R.tsv
cost_R = pd.read_csv(path + '/cost-R.tsv', sep='\t', header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
model_A.head()

,0,1,2,3
0,index,x,v,a
1,x,1,1,0
2,v,0,1,1
3,a,0,0,1


In [3]:
model_B.head()

,0,1
0,index,u
1,x,0
2,v,0
3,a,1


In [4]:
cost_Q.head()

,0,1,2,3
0,index,x,v,a
1,x,5,0,0
2,v,0,1,0
3,a,0,0,2


In [5]:
cost_R.head()

,0,1
0,index,u
1,u,5


In [6]:
# Convert dataframes to numpy arrays and remove header row
np_A = model_A.iloc[1:, 1:].values.astype(float)
np_B = model_B.iloc[1:, 1:].values.astype(float)
np_Q = cost_Q.iloc[1:, 1:].values.astype(float)
np_R = cost_R.iloc[1:, 1:].values.astype(float)

# Implement the discrete-time LQR equation to find P (Ricatti equation)
  # P = A'PA - A'PB(R + B'PB)^-1 B'PA + Q
  # Use iterative approach to solve for P

np_P = np_Q # Initial guess for P

# Iterate until P converges
for i in range(100): # Iterate for 100 steps
  np_P_next = np_A.T @ np_P @ np_A - np_A.T @ np_P @ np_B @ np.linalg.inv(np_R + np_B.T @ np_P @ np_B) @ np_B.T @ np_P @ np_A + np_Q
  if np.allclose(np_P, np_P_next):
    break
  np_P = np_P_next

# Compute the control matrix K
K_matrix = np.linalg.inv(np_R + np_B.T @ np_P @ np_B) @ np_B.T @ np_P @ np_A

# Print the computed K matrix
print("Computed Control Matrix K:")
print(K_matrix)

Computed Control Matrix K:
[[0.3344602  1.30445299 1.8581282 ]]


In [7]:
control_K_intended = pd.DataFrame(K_matrix, columns=['x', 'v', 'a'])
control_K_intended

,x,v,a
0,0.33446,1.304453,1.858128


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [8]:
# YOUR CHANGES HERE

control_K_intended.to_csv(path + '/control-K-intended.tsv', sep='\t', index=False)

Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [9]:
# YOUR CHANGES HERE

# Import qc-log.tsv
qc_log = pd.read_csv(path + '/qc-log.tsv', sep='\t')
qc_log.head()

,index,x,v,a,u
0,0,-5.000000,0.000000,0.000000,1.702188
1,1,-5.000000,-0.017022,1.531969,-1.263200
2,2,-5.018724,1.452683,0.548285,-1.249321
3,3,-3.420773,1.840779,-0.521275,-0.212127
4,4,-1.395916,1.163611,-0.764317,0.452895


In [10]:
# YOUR CHANGES HERE

# Extract the state vectors (x, v, a) from the qc_log dataframe
p2_matrix = qc_log[['x', 'v', 'a']].values

# Recompute the actions using the control matrix K and the state vectors
# u = -Kx
u_check = -K_matrix @ p2_matrix.T

# Transpose u_check back to the original orientation
u_check = u_check.T

# Create a new dataframe with index and u_check
qc_check = pd.DataFrame({'index': qc_log['index'], 'u_check': u_check.flatten()})

# Display the first few rows of the recomputed actions
display(qc_check)

,index,u_check
0,0,1.672301e+00
1,1,-1.152090e+00
2,2,-1.235178e+00
3,3,-2.885016e-01
4,4,3.692013e-01
...,...,...
95,95,-5.148662e-19
96,96,2.552219e-19
97,97,3.971328e-19
98,98,1.886528e-19


Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [11]:
# YOUR CHANGES HERE

qc_check.to_csv(path + '/qc-check.tsv', sep='\t', index=False)

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [12]:
# Import control-K-intended.tsv
control_K_intended = pd.read_csv(path + '/control-K-intended.tsv', sep='\t')
control_K_intended

,x,v,a
0,0.33446,1.304453,1.858128


In [13]:
# YOUR CHANGES HERE

# Extract the state vectors (x, v, a) and the actual actions (u) from the qc_log dataframe
X_actual = qc_log[['x', 'v', 'a']].values
u_actual = qc_log['u'].values

# Create a Linear Regression model
model_P3 = LinearRegression()

# Fit the model to the data. Note: In LQR, u = -Kx, so we regress u on -X
model_P3.fit(-X_actual, u_actual)

# The coefficients of the linear regression model are the elements of the control_K_actual_coefficients matrix
control_K_actual_coefficients = model_P3.coef_

# Create a dataframe for K_actual with the same format as control_K_intended
control_K_actual = pd.DataFrame([control_K_actual_coefficients], columns=['x', 'v', 'a'])

# Display the computed K_actual matrix
display(control_K_actual)

,x,v,a
0,0.340438,1.30012,1.950117


Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [14]:
# YOUR CHANGES HERE

control_K_actual.to_csv(path + '/control-K-actual.tsv', sep='\t', index=False)

Submit "control-K-actual.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [15]:
# YOUR CHANGES HERE

# Import qc-train.tsv
qc_train = pd.read_csv(path + '/qc-train.tsv', sep='\t')
qc_train.head()

,index,x,v,a,u
0,0,-5.000000,0.000000,0.000000,1.729856
1,1,-5.000000,-0.017299,1.556871,-1.311911
2,2,-5.019028,1.476577,0.531837,-1.198850
3,3,-3.394793,1.846154,-0.493944,-0.297565
4,4,-1.364024,1.195267,-0.811147,0.472619


In [16]:
# YOUR CHANGES HERE

# Prepare the data for linear regression
# Predict the next state (x_t+1) based on the current state (x_t) and action (u_t)
# The data needs to be shifted so that for each row, we have x_t, u_t, and x_t+1

# Drop the last row of qc_train for the current state and action (x_t, u_t)
X_u = qc_train.iloc[:-1][['x', 'v', 'a', 'u']].values

# Drop the first row of qc_train for the next state (x_t+1)
X_next = qc_train.iloc[1:][['x', 'v', 'a']].values

# Create a Linear Regression model
model_P4 = LinearRegression()

# Fit the model to predict X_next from X_u
model_P4.fit(X_u, X_next)

# The coefficients of the linear regression model represent the concatenated [A | B] matrix
AB_matrix = model_P4.coef_

# Separate the A and B matrices
# A is the first 3 columns of AB_matrix
A_new = AB_matrix[:, :3]

# B is the last column of AB_matrix
B_new = AB_matrix[:, 3].reshape(-1, 1)

# Create dataframes for A_new and B_new with appropriate column names for saving
model_A_new = pd.DataFrame(A_new, columns=['x', 'v', 'a'])
model_B_new = pd.DataFrame(B_new, columns=['u'])

# Display the computed A_new and B_new matrices
print("Computed A matrix:")
display(model_A_new)
print("\nComputed B matrix:")
display(model_B_new)

Computed A matrix:


,x,v,a
0,1.000000e+00,1.100000e+00,6.106227e-16
1,-1.706145e-16,9.000000e-01,9.500000e-01
2,-1.487819e-16,-7.771561e-16,1.100000e+00



Computed B matrix:


,u
0,5.551115e-17
1,-1.000000e-02
2,9.000000e-01


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [17]:
# YOUR CHANGES HERE

model_A_new.to_csv(path + '/model-A-new.tsv', sep='\t', index=False)
model_B_new.to_csv(path + '/model-B-new.tsv', sep='\t', index=False)

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

None.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

None.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

I used Gemini via Colab to explain my code to understand where fixes should be implemented based on errors that were output.